## Representing the Game State in Code

The game state object needs to enforce all of the rules of the game, and represent all of the information describing a 
single configuration of the game at a specific point in time. (The bottom right corner should be a blocked cell)

In [1]:
call_counter = 0

In [2]:
from copy import deepcopy

xlim, ylim = 2, 3  # board dimensions

# The eight movement directions possible for a chess queen
RAYS = [(1, 0), (1, -1), (0, -1), (-1, -1),
        (-1, 0), (-1, 1), (0, 1), (1, 1)]

class GameState:
    
    def __init__(self):
        self._board = [[0] * ylim for _ in range(xlim)]
        self._board[-1][-1] = 1  # block lower-right corner
        self._parity = 0
        self._player_locations = [None, None]
        
    def actions(self):
        """ Return a list of legal actions for the active player """
        
        return self.liberties(self._player_locations[self._parity])
    
    def player(self):
        """ Return the id of the active player """
        
        return self._parity
    
    def result(self, action):
        """ Return a new state that results from applying the given
        action in the current state"""
        
        newBoard = deepcopy(self)
        newBoard._board[action[0]][action[1]] = 1
        newBoard._player_locations[self._parity] = action
        newBoard._parity ^= 1
        return newBoard
    
    def terminal_test(self):
        """ return True if the current state is terminal,
        and False otherwise"""
        
        global call_counter
        call_counter =+ 1
        
        return (not self._has_liberties(self._parity)
            or not self._has_liberties(1 - self._parity))
    
    def utility(self, player_id):
        """ return +inf if the game is terminal and the
        specified player wins, return -inf if the game
        is terminal and the specified player loses, and
        return 0 if the game is not terminal
        """
        if not self.terminal_test(): return 0
        player_id_is_active = (player_id == self.player())
        active_has_liberties = self._has_liberties(self.player())
        active_player_wins = (active_has_liberties == player_id_is_active)
        return float("inf") if active_player_wins else float("-inf")
    
    def liberties(self, loc):
        """ Return a list of all open cells in the
        neighborhood of the specified location.  The list 
        should include all open spaces in a straight line
        along any row, column or diagonal from the current
        position. (Tokens CANNOT move through obstacles
        or blocked squares in queens Isolation.)
        
        Note: if loc is None, then return all empty cells
        on the board
        """        
        if loc is None: return self._get_blank_spaces()
        moves = []
        for dx, dy in RAYS:  # check each movement direction
            _x, _y = loc
            while 0 <= _x + dx < xlim and 0 <= _y + dy < ylim:
                _x, _y = _x + dx, _y + dy
                if self._board[_x][_y]:  # stop at any blocked cell
                    break
                moves.append((_x, _y))
        return moves
    
    def _has_liberties(self, player_id):
        """ Check to see if the specified player has any liberties """   
        return any(self.liberties(self._player_locations[player_id]))

    def _get_blank_spaces(self):
        """ Return a list of blank spaces on the board."""
        return [(x, y) for y in range(ylim) for x in range(xlim)
                if self._board[x][y] == 0]


### Test Graph Class

In [3]:
print("Creating empty game board...")
g = GameState()

print("Checking active player on an empty board...")
if g.player() != 0:
    print("Failed\n Uh Oh! Your game did not return player " +
          "id 0 on an empty board.")
else:
    print("Passed.")

print("Checking terminal test on an empty board...")
if g.terminal_test() != False:
    print("Failed\n Uh Oh! Your game marked an empty game state as terminal.")
else:
    print("Passed.")
    
print("Checking liberties on an empty board...")
p1_liberties = g.liberties(None)
if len(p1_liberties) != 5:
    print("Failed\n Uh oh! Your game did not return 5 empty " +
          "cell locations as liberties on an empty board.")
else:
    print("Passed.")

print("Getting legal moves for player 1...")
p1_empty_moves = g.actions()
print("Found {} legal moves.".format(len(p1_empty_moves or [])))

print("Applying move {} for player 1...".format(p1_empty_moves[0]))
g1 = g.result(p1_empty_moves[0])

print("Getting legal moves for player 2...")
p2_empty_moves = g1.actions()
if len(p2_empty_moves) != 4:
    print("Failed\n  Uh oh! Your game did not return the expected " +
          "number of actions for player 2!")
else:
    print("Passed.")

print("\nPlaying a full game")
for _ in range(5):
    if g.terminal_test(): break
    g = g.result(g.actions()[0])

print("Checking terminal test on a terminal board...")
if g.terminal_test() != True:
    print("Failed\n  Uh oh! Your game did not correctly evalute " +
          "a terminal game state as terminal!")
else:
    print("Passed.")

Creating empty game board...
Checking active player on an empty board...
Passed.
Checking terminal test on an empty board...
Passed.
Checking liberties on an empty board...
Passed.
Getting legal moves for player 1...
Found 5 legal moves.
Applying move (0, 0) for player 1...
Getting legal moves for player 2...
Passed.

Playing a full game
Checking terminal test on a terminal board...
Passed.


## Minimax Decision: Choosing the Best Branch

The minimax_decision() function should implement the eponymous procedure from the pseudocode. 
It should loop over the legal moves from the current state and return the move that has the highest score. 
The scores are determined by mutually recursive calls between the min and max value helper functions until a 
terminal state is reached, and propagated back up the tree as the call stack unwinds.

In [4]:
def min_value(gameState):
    """ Return the game state utility if the game is over,
    otherwise return the minimum value over all legal successors"""
    # always want the terminal state utility AKA the max value, so 0 does not change to a 1
    if gameState.terminal_test():
        return gameState.utility(0)
    
    v = float("inf")
    # uses current location to get all possible moves
    for a in gameState.actions():
        v = min(v, max_value(gameState.result(a)))

    return v


def max_value(gameState):
    """ Return the game state utility if the game is over,
    otherwise return the maximum value over all legal successors"""
    if gameState.terminal_test():
        return gameState.utility(0)
    
    v = float("-inf")
    # uses current location to get all possible moves
    for a in gameState.actions():
        v = max(v, min_value(gameState.result(a)))

    return v

In [5]:
def minimax_decision(gameState):
    """ Return the move along a branch of the game tree that
    has the best possible value.  A move is a pair of coordinates
    in (column, row) order corresponding to a legal move for
    the searching player."""
    # initiates the recursion
    best_score = float("-inf")
    best_action = None
    
    for a in gameState.actions():
        v = min_value(gameState.result(a))
        
        if v > best_score:
            best_score = v
            best_action = a
            
    return best_action

### Test Minimax

In [6]:
best_moves = {(0, 0), (0, 2), (1, 0)}
rootNode = GameState()
minimax_move = minimax_decision(rootNode)

print("Best move choices: {}".format(list(best_moves)))
print("Your code chose: {}".format(minimax_move))

if minimax_move in best_moves:
    print("That's one of the best move choices. Looks like your minimax-decision function worked!")
else:
    print("Uh oh...looks like there may be a problem.")

Best move choices: [(1, 0), (0, 2), (0, 0)]
Your code chose: (0, 0)
That's one of the best move choices. Looks like your minimax-decision function worked!


## Adding a Depth Limit

Add a new parameter named depth to each of the minimax functions, then update all of the function calls to pass the depth Aparameter to the next function


Todo:
1. Incorporate the `depth` parameter into each function
2. Update all recursive calls to pass the depth parameter
3. Add a new conditional to cut off search when the depth limit is reached

In [7]:
def minimax_decision(gameState, depth):
    """ Return the move along a branch of the game tree that
    has the best possible value.  A move is a pair of coordinates
    in (column, row) order corresponding to a legal move for
    the searching player."""
    # initiates the recursion
    best_score = float("-inf")
    best_action = None
    
    for a in gameState.actions():
        v = min_value(gameState.result(a), depth)
        
        if v > best_score:
            best_score = v
            best_action = a
            
    return best_action

def min_value(gameState, depth):
    """ Return the game state utility if the game is over,
    otherwise return the minimum value over all legal successors"""
    # always want the terminal state utility AKA the max value, so 0 does not change to a 1
    if gameState.terminal_test():
        return gameState.utility(0)
    
    if depth == 0:
        return 0
    
    v = float("inf")
    # uses current location to get all possible moves
    for a in gameState.actions():
        v = min(v, max_value(gameState.result(a), depth - 1))

    return v


def max_value(gameState, depth):
    """ Return the game state utility if the game is over,
    otherwise return the maximum value over all legal successors"""
    if gameState.terminal_test():
        return gameState.utility(0)
    
    if depth == 0:
        return 0
    
    v = float("-inf")
    # uses current location to get all possible moves
    for a in gameState.actions():
        v = max(v, min_value(gameState.result(a), depth - 1))

    return v

### Test Depth Limit

In [8]:
call_counter = 0
depth_limit = 1
expected_node_count = 5
rootNode = GameState()
_ = minimax_decision(rootNode, depth_limit)

print("Expected node count: {}".format(expected_node_count))
print("Your node count: {}".format(call_counter))

if call_counter == expected_node_count:
    print("That's right! Looks like your depth limit is working!")
else:
    print("Uh oh...looks like there may be a problem.")

Expected node count: 5
Your node count: 1
Uh oh...looks like there may be a problem.


## Adding an Evaluation Function

We can improve the performance of depth limited search using the concept of an "evaluation function" (also called a heuristic function)

Todo:
1. Implement the my_moves() function
2. Change the value returned when the depth cutoff is Reached to call and return the score from my_moves()

In [9]:
def my_moves(gameState):
    location = gameState._player_locations[0]
    return len(gameState.liberties(location))

### Test Evaluation Function

In [10]:
def minimax_decision(gameState, depth):
    """ Return the move along a branch of the game tree that
    has the best possible value.  A move is a pair of coordinates
    in (column, row) order corresponding to a legal move for
    the searching player."""
    # initiates the recursion
    best_score = float("-inf")
    best_action = None
    
    for a in gameState.actions():
        v = min_value(gameState.result(a), depth)
        
        if v > best_score:
            best_score = v
            best_action = a
            
    return best_action

def min_value(gameState, depth):
    """ Return the game state utility if the game is over,
    otherwise return the minimum value over all legal successors"""
    # always want the terminal state utility AKA the max value, so 0 does not change to a 1
    if gameState.terminal_test():
        return gameState.utility(0)
    
    if depth == 0:
        return my_moves(gameState)
    
    v = float("inf")
    # uses current location to get all possible moves
    for a in gameState.actions():
        v = min(v, max_value(gameState.result(a), depth - 1))

    return v


def max_value(gameState, depth):
    """ Return the game state utility if the game is over,
    otherwise return the maximum value over all legal successors"""
    if gameState.terminal_test():
        return gameState.utility(0)
    
    if depth == 0:
        return my_moves(gameState)
    
    v = float("-inf")
    # uses current location to get all possible moves
    for a in gameState.actions():
        v = max(v, min_value(gameState.result(a), depth - 1))

    return v

In [11]:
depth_limit = 1
rootNode = GameState()
tests = [((0, 0), 2), ((0, 1), 3), ((0, 2), 1), ((1, 0), 2), ((1, 1), 3)]

if all(min_value(rootNode.result(a), depth_limit) == v for a, v in tests):
    print("Good job!")
else:
    print("Uh oh!\n Looks like one or more of the values didn't match.")

Good job!


## Iterative Deepening
Iterative deepening is a search technique that allows minimax-style search functions to return an approximate solution when computational resources are bounded. The basic idea is to start with a small depth-limited search, and grow the depth limit until the resource limit (usually search time) expires.

Todo: 
Implement a function that calls minimax_decision for each depth from 1...depth_limit (inclusive of both endpoints)

In [12]:
def get_action(gameState, depth_limit):  
    for depth in range(1, depth_limit + 1):
        a = minimax_decision(gameState, depth)
        
    return a
        

### Test Iterative Deepening

In [13]:
call_counter = 0
depth_limit = 2
expected_node_count = 30
rootNode = GameState()
get_action(rootNode, depth_limit)

print("Expected node count: {}".format(expected_node_count))
print("Your node count: {}".format(call_counter))

if call_counter == expected_node_count:
    print("That's right! Looks like your depth limit is working!")
else:
    print("Uh oh...looks like there may be a problem.")

Expected node count: 30
Your node count: 1
Uh oh...looks like there may be a problem.


## Alpha-Beta Pruning

In [16]:
def alpha_beta_search(gameState):
    """ Return the move along a branch of the game tree that
    has the best possible value.  A move is a pair of coordinates
    in (column, row) order corresponding to a legal move for
    the searching player."""
    # this now initiates the recursion instead of the other
    
    alpha = float("-inf")
    beta = float("inf")
    best_score = float("-inf")
    best_move = None

    for a in gameState.actions():
        v = min_value(gameState.result(a), alpha, beta)
        alpha = max(alpha, v)
        if v > best_score:
            best_score = v
            best_move = a
    return best_move

def min_value(gameState, alpha, beta):
    """ Return the value for a win (+1) if the game is over,
    otherwise return the minimum value over all legal child
    nodes.
    """
    if gameState.terminal_test():
        return gameState.utility(0)
    
    v = float("inf")
    for a in gameState.actions():        
        v = min(v, max_value(gameState.result(a), alpha, beta))
        if alpha >= v:
            return v  
        beta = min(beta, v)     
    return v

def max_value(gameState, alpha, beta):
    """ Return the value for a loss (-1) if the game is over,
    otherwise return the maximum value over all legal child
    nodes.
    """
    if gameState.terminal_test():
        return gameState.utility(0)
    
    v = float("-inf")
    for a in gameState.actions():       
        v = max(v, min_value(gameState.result(a), alpha, beta))
        if v >= alpha:
            return v  
        alpha = max(alpha, v)           
    return v

### Test Alpha-Beta Pruning

In [17]:
call_counter = 0
expected_node_count = 57
rootNode = GameState()
alpha_beta_search(rootNode)

print("Expected node count: {}".format(expected_node_count))
print("Your node count: {}".format(call_counter))

if call_counter == expected_node_count:
    print("That's right! Looks like your alpha-beta pruning is working!")
else:
    print("Uh oh...looks like there may be a problem.")


Expected node count: 57
Your node count: 1
Uh oh...looks like there may be a problem.
